In [203]:
import torch
from torch.utils.data import TensorDataset, DataLoader
import torch.nn as nn
import torch.optim as optim
import numpy as np
import pandas as pd

In [204]:
pathogen = 3
featnum = 5

pathogenfilenamedir_test = f'/home/yhchoi/FSTrain_0205/samplevectors/{pathogen}/filenames_test.pt'
pathogenfilenames_test = torch.load(pathogenfilenamedir_test)

In [205]:
internal_minmax = pd.read_excel('/home/yhchoi/FSTrain_0205/extdata_labels/sinchon_minmax.xlsx')
internal_min_dict = internal_minmax.iloc[0].to_dict()
internal_max_dict = internal_minmax.iloc[1].to_dict()

In [206]:
all_clinical_features = pd.read_excel('/home/yhchoi/FSTrain_0205/XGBoost/features_0228/feautres.xlsx')

acfdict = dict(zip(all_clinical_features['Unnamed: 0'], all_clinical_features['Unnamed: 1']))
acfdict_bykey = {1:[], 2:[], 3:[], 4:[], 5:[], 6:[]}

for key, value in acfdict.items():
    acfdict_bykey[value].append(key)

acfdict_bykey[6] = [item for item in acfdict_bykey[6] if item not in ['serum_d_dimer', 'serum_iCa', 'serum_iMg', 'serum_CK_MB', 'serum_TG', 'serum_HDL', 'serum_amylase', 'serum_lipase', 'serum_TSH', 'serum_T3', 'serum_fT4', 'serum_vitB12', 'serum_delta_neutrophil']]
clinicals = pd.read_csv('/home/yhchoi/24BMTesting_ExtData/clinical/external_clinical_data.csv')

columns = list(clinicals.columns)
columns[2] = 'Pathogen'
columns[3] = 'Prognosis'
clinicals.columns = columns
columns_to_exclude = columns[:4]

no_minmax = []
def minmax_scale(column):
    if column.name in columns_to_exclude:
        return column
    try:
        return (column - internal_min_dict[column.name]) / (internal_max_dict[column.name] - internal_min_dict[column.name])
    except:
        no_minmax.append(column.name)
        return column

scaled_df = clinicals.apply(minmax_scale)

scaled_df.fillna(-1, inplace=True)
# clinicals.to_csv('/home/yhchoi/FSTrain_0205/XGBoost/features_0228/clinical_features.csv', index=False)
exttestdatasets = [scaled_df[['Filename', 'Prognosis', 'Pathogen'] + acfdict_bykey[i]] for i in range(1, 7)]


In [207]:
datasetperpathogen_test = exttestdatasets[featnum]
datasetperpathogen_test = datasetperpathogen_test[datasetperpathogen_test['Pathogen'] == pathogen]
filenamelistperpathogen_test = datasetperpathogen_test['Filename'].tolist()

In [208]:
filenameindices_test = [filenamelistperpathogen_test.index(item) for item in pathogenfilenames_test]

In [209]:
data_test = exttestdatasets[featnum]
data_test = data_test[data_test['Pathogen'] == pathogen]
feature_columns = acfdict_bykey[featnum + 1]
# Convert data to PyTorch tensors
X_test = torch.tensor(data_test[feature_columns].values, dtype=torch.float32)
y_test = torch.tensor(data_test['Prognosis'].values, dtype=torch.long)
all_featvecs_test = X_test[filenameindices_test]

In [210]:
torch.save(all_featvecs_test, f'/home/yhchoi/FSTrain_0205/samplevectors/{pathogen}/clinical_featnum{featnum}_raw_featurevectors_test.pt')